TED talk text generator

Generate using a character based model.

The code is this notebook is a partial modification of several tutorials, blog posts and YouTube videos. The main sources include:
* [Text Generation With LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) by Dr. Jason Brownlee
* [Text Generation with Python and TensorFlow/Keras](https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/) by Dan Nelson
* YouTube: [167 - Text prediction using LSTM (English text)](https://youtu.be/zyCpntcVKSo?t=2) by Sreenivas B.


In [34]:
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
#   print('and then re-execute this cell.')
# else:
#   print(gpu_info)

In [35]:
# from psutil import virtual_memory
# ram_gb = virtual_memory().total / 1e9
# print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

# if ram_gb < 20:
#   print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
#   print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
#   print('re-execute this cell.')
# else:
#   print('You are using a high-RAM runtime!')

In [36]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
import random

In [37]:
# # Attach to my Google drive so I can save the csv file later
# from google.colab import drive
# drive.mount('/content/drive')

In [38]:
#tag_name = 'all_transcripts'
#tag_name = 'brain'
#tag_name = 'business'
#tag_name = 'creativity'
#tag_name = 'culture'
tag_name = 'psychology'
#tag_name = 'science'

lower_corpus = False

if lower_corpus:
    corpus_file_name = tag_name + '_lowercase'
else:
    corpus_file_name = tag_name + '_sentence_case'

# This switch makes it easier running locally or in a Google Colab environment
colab = False

if colab:
    root_path = '/content/drive/MyDrive/ted/'
else:
    root_path = '../data/'

In [39]:
corpus_file_name

'psychology_sentence_case'

In [40]:
# if on Google Colab, load transcripts file.
# with open(f'/content/drive/MyDrive/ted/{corpus_file_name}.txt') as f:
#   corpus = f.read()
# f.close()

# if run locally:
with open(f'{root_path}{corpus_file_name}.txt', encoding='utf-8') as f:
    corpus = f.read()
f.close()

In [41]:
corpus[0:500]

"So I want to start by offering you a free no-tech life hack, and all it requires of you is  that you change your posture for two minutes. But before I give it away, I want to ask you to right now do a little audit of your body and what you're doing with your body. So how many of you are sort of making yourselves smaller? Maybe you're hunching, crossing your legs, maybe wrapping your ankles. Sometimes we hold onto our arms like this. Sometimes we spread out.  I see you. So I want you to pay atten"

The code below is used in almost every article I have seen about LSTM for text generation, but this snippet is adapted from :
https://www.kaggle.com/mrisdal/intro-to-lstms-w-keras-gpu-for-text-generation/

In [42]:
all_chars = sorted(list(set(corpus)))

#remove most punctuation, but leave spaces, exclamation points, and periods
list_to_remove = ['"', '&', '-', ',', "'", '/', ';',  '—']

chars = [character for character in all_chars if character not in list_to_remove]
print(chars)
# hat tip to this post for the idea of removing one list from another
# https://www.geeksforgeeks.org/python-remove-all-values-from-a-list-present-in-other-list/

[' ', '!', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [43]:
# remove all unwanted characters from the original corpus
corpus_list = [character for character in corpus if character not in list_to_remove]

# Re-join all letters back into a single string as the updated corpus
corpus = ''.join(corpus_list)

In [44]:
print('Number of unique characters in the corpus:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))

indices_char = dict((i, c) for i, c in enumerate(chars))

Number of unique characters in the corpus: 65


In [45]:
# Show character to index mapping
print (char_indices)

{' ': 0, '!': 1, '.': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, '?': 13, 'A': 14, 'B': 15, 'C': 16, 'D': 17, 'E': 18, 'F': 19, 'G': 20, 'H': 21, 'I': 22, 'J': 23, 'K': 24, 'L': 25, 'M': 26, 'N': 27, 'O': 28, 'P': 29, 'Q': 30, 'R': 31, 'S': 32, 'T': 33, 'U': 34, 'V': 35, 'W': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [46]:
# Show index to character mapping
print(indices_char)

{0: ' ', 1: '!', 2: '.', 3: '0', 4: '1', 5: '2', 6: '3', 7: '4', 8: '5', 9: '6', 10: '7', 11: '8', 12: '9', 13: '?', 14: 'A', 15: 'B', 16: 'C', 17: 'D', 18: 'E', 19: 'F', 20: 'G', 21: 'H', 22: 'I', 23: 'J', 24: 'K', 25: 'L', 26: 'M', 27: 'N', 28: 'O', 29: 'P', 30: 'Q', 31: 'R', 32: 'S', 33: 'T', 34: 'U', 35: 'V', 36: 'W', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}


In [47]:
print (f'The input corpus contains {len(corpus):,} characters.')

The input corpus contains 81,675 characters.


The model will use arbitrary length of characters, e.g. 40, and then predict the next character that will appear (the 41st).

I would like to change this sequence length in different models, to see how it might affect the model's performance.

Larger might be better?

In [48]:
sequence_length = 40

Loop through the corpus, creating segments of 40 characters, plus a segment of a single character that would appear after it. These segments are then converted into their respective digits, and loaded into X

In addition, create a list of target values. The target, y, is a single text character, and is also converted into a corresponding numeric value.

In [49]:
corpus[0:50]

'So I want to start by offering you a free notech l'

In [50]:
corpus[1:41]

'o I want to start by offering you a free'

In [51]:
corpus[41]

' '

In [52]:
# For future modeling, we could try to offset the sequences by more than one
# letter. For now the model will look at every sequence right after another.
step = 1

X_numeric_list = []
y_numeric_list = []


for i in range (0, len(corpus) - sequence_length, step):
    # To find X when still as characters, loop through and extract
    # a sequence for example, from the 2nd to the 42nd, as in corpus[1:41] since 
    # the string index is zero-based
    X_char_sequence = corpus[i:i + sequence_length]  #exclusive
    y_char = corpus[i + sequence_length]

    # Convert the X character sequence into a list of integers, using the 
    # dictionary created above.
    X_numeric_list.append( [char_indices[letter] for letter in X_char_sequence])
    # Also convert target letter y to it's corresponding numeric value in the
    # dictionary
    y_numeric_list.append(char_indices[y_char])



In [53]:
# Look at the first two converted sequences
print ( X_numeric_list[0])
print ( X_numeric_list[1])


[32, 53, 0, 22, 0, 61, 39, 52, 58, 0, 58, 53, 0, 57, 58, 39, 56, 58, 0, 40, 63, 0, 53, 44, 44, 43, 56, 47, 52, 45, 0, 63, 53, 59, 0, 39, 0, 44, 56, 43]
[53, 0, 22, 0, 61, 39, 52, 58, 0, 58, 53, 0, 57, 58, 39, 56, 58, 0, 40, 63, 0, 53, 44, 44, 43, 56, 47, 52, 45, 0, 63, 53, 59, 0, 39, 0, 44, 56, 43, 43]


In [54]:
print ( y_numeric_list[0:2])

[43, 0]


In [55]:
indices_char[51]

'm'

In [56]:
corpus[0:43]

'So I want to start by offering you a free n'

This example is a little hard to follow, since we happened to be in the middle of the word 'been'. The letter e is mapped to 51.

However the beginning of the sequence has been shifted by one as well.

In [57]:
print (f'There are {len(X_numeric_list):,} entries in X_numeric_list') 

There are 81,635 entries in X_numeric_list


An LSTM model needs the X data to be in 3 dimensions:
* Samples (number of rows)
* Time steps (this implies time series. In this example this corresponds to the sequence_length)
* Features (the target, or y variable)

This [blog post by Dr. John Brownlee](https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/) contains a longer explanation, which I paraphrased here.

In [58]:
# reshape X to be [samples, time steps, features]
# Samples is the total number of 40 character text fragments created above
# Time step is the sequence length, which is this case is 40, but might change
# in other model variations
# Feature is one, since the model is predicting one character at a time
X = np.reshape(X_numeric_list, (len(X_numeric_list), sequence_length, 1))

In [59]:
X.shape

(81635, 40, 1)

In [60]:
# normalize/scale the data by dividing by the length of the character list.
X = X / len(chars)

In [61]:
# Look at the first value of X
X[0]

array([[0.49230769],
       [0.81538462],
       [0.        ],
       [0.33846154],
       [0.        ],
       [0.93846154],
       [0.6       ],
       [0.8       ],
       [0.89230769],
       [0.        ],
       [0.89230769],
       [0.81538462],
       [0.        ],
       [0.87692308],
       [0.89230769],
       [0.6       ],
       [0.86153846],
       [0.89230769],
       [0.        ],
       [0.61538462],
       [0.96923077],
       [0.        ],
       [0.81538462],
       [0.67692308],
       [0.67692308],
       [0.66153846],
       [0.86153846],
       [0.72307692],
       [0.8       ],
       [0.69230769],
       [0.        ],
       [0.96923077],
       [0.81538462],
       [0.90769231],
       [0.        ],
       [0.6       ],
       [0.        ],
       [0.67692308],
       [0.86153846],
       [0.66153846]])

Even thought the values for the target variable are integers, they are essentially labels for the predicted characters. They are one-hot encoded using the to_categorical function.

In [62]:
y = to_categorical(y_numeric_list)
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

---
### Create LSTM Model

This [blog post](https://towardsdatascience.com/choosing-the-right-hyperparameters-for-a-simple-lstm-using-keras-f8e9ed76f046) has a rule of thumb of how many nodes to include in an LSTM.

In [63]:
hidden_nodes = int(2/3 * (len(chars) * sequence_length))
print(f"The number of hidden nodes is {hidden_nodes}.")

The number of hidden nodes is 1733.


Use of ModelCheckPoint to save the model's progress example as described in the [TensorFlow documentation.](https://www.tensorflow.org/tutorials/keras/save_and_load#checkpoint_callback_options)

In [64]:
#Set a path to save the checkpoint files for this model
checkpoint_path = f'{root_path}{corpus_file_name}/{corpus_file_name}.ckpt'
checkpoint_dir = f'{root_path}{corpus_file_name}/'

# Create a callback that saves the model's weights every 50 fits
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_freq=50)

In [65]:
# # define the LSTM model
# model = Sequential()
# model.add(LSTM(400, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.3))
# model.add(LSTM(128))
# model.add(Dropout(0.3))
# model.add(Dense(y.shape[1], activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy','Recall','Precision'])

# # Save the weights using the `checkpoint_path` format
# model.save_weights(checkpoint_path.format(epoch=0))

In [67]:
model = load_model(f'{root_path}/models/ted_model_{tag_name}')
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 40, 400)           643200    
_________________________________________________________________
dropout_6 (Dropout)          (None, 40, 400)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)               270848    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 65)                8385      
Total params: 922,433
Trainable params: 922,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, y, epochs=50, batch_size=100, callbacks=[cp_callback])

Epoch 1/50
 50/817 [>.............................] - ETA: 2:23 - loss: 1.5006
Epoch 00001: saving model to ../data/psychology_sentence_case\psychology_sentence_case.ckpt


INFO:tensorflow:Assets written to: ../data/psychology_sentence_case\psychology_sentence_case.ckpt\assets


INFO:tensorflow:Assets written to: ../data/psychology_sentence_case\psychology_sentence_case.ckpt\assets


100/817 [==>...........................] - ETA: 4:04 - loss: 1.5123
Epoch 00001: saving model to ../data/psychology_sentence_case\psychology_sentence_case.ckpt


INFO:tensorflow:Assets written to: ../data/psychology_sentence_case\psychology_sentence_case.ckpt\assets


...

INFO:tensorflow:Assets written to: ../data/psychology_sentence_case\psychology_sentence_case.ckpt\assets


512/817 [=================>............] - ETA: 2:47 - loss: 1.3259
Epoch 00015: saving model to ../data/psychology_sentence_case\psychology_sentence_case.ckpt


INFO:tensorflow:Assets written to: ../data/psychology_sentence_case\psychology_sentence_case.ckpt\assets


INFO:tensorflow:Assets written to: ../data/psychology_sentence_case\psychology_sentence_case.ckpt\assets


523/817 [==================>...........] - ETA: 2:46 - loss: 1.3254

In [ ]:
# Save the model to my Google Drive so I can load it later from another notebook
#model.save(f'/content/drive/MyDrive/ted/models/ted_model_{tag_name}')

model.save(f'{root_path}/ted_model_{tag_name}')